# Spark in Action - Chapter 3 Scala Version - Lab 320

In [ ]:
import java.time.LocalDate
import java.time.format.DateTimeFormatter
import org.apache.spark.sql.{functions => F}
import org.apache.spark.sql.{Dataset, Row, Encoders, SparkSession}

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.41:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1667057073964)
SparkSession available as 'spark'


import java.time.LocalDate
import java.time.format.DateTimeFormatter
import org.apache.spark.sql.{functions=>F}
import org.apache.spark.sql.{Dataset, Row, Encoders, SparkSession}


In [ ]:
// Creates a session on a local master
val spark = SparkSession
                .builder
                .appName("CSV to dataframe to Dataset<Book> and back")
                .master("local[*]")
                .getOrCreate

22/10/29 17:24:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6dd36ebe


In [ ]:
case class Book(authorId:Int, title:String, releaseDate:LocalDate, link:String, id:Int=0)

defined class Book


In [ ]:
/**
* This is a mapper class that will convert a Row to an instance of Book.
* You have full control over it - isn't it great that sometimes you have
* control?
*
* @author rambabu.posa
*/
def rowToBook(row: Row): Book = {
    val dateAsString = row.getAs[String]("releaseDate")

    val releaseDate = if (dateAsString == null) {
        LocalDate.parse(
          "01/01/23",
          DateTimeFormatter.ofPattern("M/d/yy")
        )
    } else {
        LocalDate.parse(
          dateAsString,
          DateTimeFormatter.ofPattern("M/d/yy")
        )
    }
    
    val author1 = row.getAs[Int]("authorId")
    val author2 = if (Option(author1).isDefined) {
        author1
    } else {
        0
    }

    Book(
      author2,
      row.getAs[String]("title"),
      releaseDate,
      row.getAs[String]("link"),
      row.getAs[Int]("id")
    )
}

rowToBook: (row: org.apache.spark.sql.Row)Book


In [ ]:
val dateAsString = "6/17/17" //null

val releaseDate = if (dateAsString == null) {
    LocalDate.parse(
      "01/01/23",
      DateTimeFormatter.ofPattern("M/d/yy")
    )
} else {
    LocalDate.parse(
      dateAsString,
      DateTimeFormatter.ofPattern("M/d/yy")
    )
}

dateAsString: String = 6/17/17
releaseDate: java.time.LocalDate = 2017-06-17


In [ ]:
val author1 = null
val author2 = if (Option(author1).isDefined) {
    author1
} else {
    99
}

author1: Null = null
author2: Any = 99


In [ ]:
val filename = "../net.jgp.books.spark.ch03/data/books.csv"

filename: String = ../net.jgp.books.spark.ch03/data/books.csv


In [ ]:
val df = spark.read.format("csv")
            .option("inferSchema", "true")
            .option("header", "true")
            .load(filename)

df: org.apache.spark.sql.DataFrame = [id: int, authorId: int ... 3 more fields]


In [ ]:
println("*** Books ingested in a dataframe")
df.show(30)
df.printSchema()
df.count()

*** Books ingested in a dataframe
+---+--------+--------------------+-----------+--------------------+
| id|authorId|               title|releaseDate|                link|
+---+--------+--------------------+-----------+--------------------+
|  1|       1|Fantastic Beasts ...|   11/18/16|http://amzn.to/2k...|
|  2|       1|Harry Potter and ...|    10/6/15|http://amzn.to/2l...|
|  3|       1|The Tales of Beed...|    12/4/08|http://amzn.to/2k...|
|  4|       1|Harry Potter and ...|    10/4/16|http://amzn.to/2k...|
|  5|       2|Informix 12.10 on...|    4/23/17|http://amzn.to/2i...|
|  6|       2|Development Tools...|   12/28/16|http://amzn.to/2v...|
|  7|       3|Adventures of Huc...|    5/26/94|http://amzn.to/2w...|
|  8|       3|A Connecticut Yan...|    6/17/17|http://amzn.to/2x...|
| 10|       4|Jacques le Fataliste|     3/1/00|http://amzn.to/2u...|
| 11|       4|Diderot Encyclope...|       null|http://amzn.to/2i...|
| 12|    null|   A Woman in Berlin|    7/11/06|http://amzn.to/2i...|


res0: Long = 22


In [ ]:
import spark.implicits._
val bookDs:Dataset[Book] = df.map(rowToBook)

import spark.implicits._
bookDs: org.apache.spark.sql.Dataset[Book] = [authorId: int, title: string ... 3 more fields]


In [ ]:
bookDs.printSchema()

root
 |-- authorId: integer (nullable = false)
 |-- title: string (nullable = true)
 |-- releaseDate: date (nullable = true)
 |-- link: string (nullable = true)
 |-- id: integer (nullable = false)



In [ ]:
bookDs.show(5)

22/10/29 17:24:50 ERROR Executor: Exception in task 0.0 in stage 6.0 (TID 5)
java.lang.ClassCastException: $line8.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$Book cannot be cast to $line8.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$Book
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.mapelements_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.deserializetoobject_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:364)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at 

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 5) (192.168.1.41 executor driver): java.lang.ClassCastException: Book cannot be cast to Book

In [ ]:
println("*** Books are now in a dataset of books")
bookDs.show(5)
bookDs.printSchema()

In [ ]:
var df2 = bookDs.toDF

In [ ]:
df2 = df2.withColumn("releaseDateAsString",
      F.date_format(F.col("releaseDate"), "M/d/yy").as("MM/dd/yyyy"))

In [ ]:
println("*** Books are back in a dataframe")
df2.show(5, 13)
df2.printSchema()

In [ ]:
spark.stop()